# NEUS 642 - Week 6 Homework

Time for a mini analysis project!

Note that the `data` folder contains additional files than the ones we studied in class. Each file starts with an id (e.g., eno052d-a1), indicating a distinct neuron. Note also that there are pairs of files, one `pupil` and one `spikes`, for each neuron. (In practice these data were all collected simultaneously and the pupil traces are identical for each neuron here, but let's ignore that for now).

Your job: test each cell in this dataset for signficant effects of pupil on spontaneous and evoked firing rate.

## Lecture recap

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [15]:
cellid = 'eno052d-a1'
spikefn=f'data/{cellid}_spikes.csv'
pupilfn=f'data/{cellid}_pupil.csv'
print('loading', spikefn, ',', pupilfn)
spikes = np.loadtxt(spikefn, delimiter=",")
pupil = np.loadtxt(pupilfn, delimiter=",")
fs = 10

loading data/eno052d-a1_spikes.csv , data/eno052d-a1_pupil.csv


In [10]:
mean_pupil_per_trial = np.mean(pupil, axis=0)
pupil_median = np.median(mean_pupil_per_trial)
pre_stimulus_bins = int(2 * fs)
post_stimulus_bins = int(0.5 * fs)
response = spikes[pre_stimulus_bins:-post_stimulus_bins,:].mean(axis=0) * fs
spont = spikes[:pre_stimulus_bins,:].mean(axis=0) * fs

small_trials = (mean_pupil_per_trial<=pupil_median)
large_trials = (mean_pupil_per_trial>pupil_median)

In [13]:
def my_permutation_test(a,b,N=100):
    """
    use permutation to test whether the means of a and b are significantly different.
    inputs:
       a, b: 1-D distributions of data
       N: number of permutations
    outputs:
       p: p-value, how likely the absolute value of the mean difference occured by chance
    """
    # first, compute the shuffled mean differences
    shuffled_distribution = np.zeros(N)
    for i in range(N):
        all_data = np.append(a, b)
        s = np.random.permutation(all_data)
        a_shuffled = s[:len(a)]
        b_shuffled = s[len(a):]
        shuffled_distribution[i] = np.mean(a_shuffled) - np.mean(b_shuffled)
    
    # now compute p
    effective_distribution = np.append(shuffled_distribution, a.mean()-b.mean())

    total_N = len(effective_distribution)
    greater_equal_N = np.sum(np.abs(effective_distribution) >= np.abs(a.mean()-b.mean()))
    p = greater_equal_N / total_N
    
    return p

In [17]:
p_response = my_permutation_test(response[small_trials], response[large_trials], N=100)
p_spont = my_permutation_test(spont[small_trials], spont[large_trials], N=100)
print(f"cell: {cellid}  p_response: {p_response:.5f}  p_spont: {p_spont:.5f}")

cell: eno052d-a1 p_response: 0.00990 p_spont: 0.12871


## Question 1 - Load and test the dataset

Load data from each of the five neurons and report the signficance of pupil-dependent differences in mean and evoked firing rate for each neurons, similarly to what the code above does for one neuron.

## Question 2 - Plot your results

Compute the mean spont and response in large and small pupil state for each neuron. Using whatever approach you like, plot the data, indicating on the plot measures that show a significant difference due to pupil size.

An idea, if you don't already have something in mind: For each metric (spont, resposne), generate a scatter plot with small pupil value on the x axis and large pupil value on y. Color the dots as to whether their difference is significant, according to our permutation test.